In [167]:
!pip install transformers

In [168]:
import os

import random
import numpy as np

import pandas as pd
import torch
from torch.utils.data import Dataset, random_split
from transformers import (
    AdamW,
    Trainer,
    TrainingArguments,
    AutoTokenizer,
    AutoModelForSequenceClassification,
    get_cosine_schedule_with_warmup,
)
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import gc

from transformers import pipeline
import re

In [169]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Load the training data
data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')


In [170]:
data

,OriginalPage,Text
0,1,I sit down alone at the appointed table and ta...
1,2,I plunged for the last time. The few remaining...
2,3,"At my meeting with Clement yesterday, he had\n..."
3,4,And I really think I would have preferred the\...
4,5,I hated my eye for being caught by what didn�t...
...,...,...
70,71,But next day that religious fellow�s head\ndra...
71,72,Of course I was sorry to say good-bye to old\n...
72,73,"All the artist in me flared up. After all, my\..."
73,74,"photographs of young and laughing athletes,\nl..."


In [171]:
split_data = pd.DataFrame(columns=['text', 'OriginalPage'])

In [172]:
for _, row in data.iterrows():
    # Split the text into two equal parts
    text1, text2 = row['Text'][:len(row['Text']) // 2], row['Text'][len(row['Text']) // 2:]
    # Add the split texts to the dataframe as separate rows
    split_data = split_data.append({'text': text1, 'OriginalPage': row['OriginalPage']}, ignore_index=True)
    split_data = split_data.append({'text': text2, 'OriginalPage': row['OriginalPage']}, ignore_index=True)
    
split_data=split_data.sample(frac=1)
# Print the resulting dataframe
split_data

,text,OriginalPage
140,But next day that religious fellow�s head\ndra...,71
39,"\nUnderneath this stone, he had said, doth lie...",20
119,"-scientific seas no\nmore, at least. There had...",60
3,"as far as I could see, and I\nrecalled the sta...",2
44,I hate seeing things like this in the paper. B...,23
...,...,...
135,"if you\nlike, I could hope to cope. I was the...",68
4,"At my meeting with Clement yesterday, he had\n...",3
17,hould enjoy it\nas I got used to it. The bell ...,9
118,I had sufficient knowledge to realise that I h...,60


In [173]:
def remove_newlines(text):
    return re.sub(r'\n', '', text)

In [174]:
# Preprocess the text data by lowercasing and removing punctuation
split_data['text'] = split_data['text'].apply(remove_newlines)

In [175]:
X = split_data['text']
X = X.str.lower().str.replace(r'[^\w\s]', '')
y = split_data['OriginalPage']

In [176]:
vectorizer = TfidfVectorizer()
tfidf_data = vectorizer.fit_transform(X).toarray()
features = vectorizer.get_feature_names_out()
tfidf_data = pd.DataFrame(tfidf_data, columns=features)

In [188]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import  MultinomialNB
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV

In [189]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest=train_test_split(tfidf_data, y, test_size=0.1, random_state=11)

In [178]:
# !pip install lazypredict

In [190]:
from lazypredict.Supervised import LazyRegressor
from sklearn.utils import shuffle
import numpy as np

reg = LazyRegressor(verbose=0,ignore_warnings=False, custom_metric=None )
models,predictions = reg.fit(xtrain, xtest, ytrain, ytest)

 55%|█████▍    | 23/42 [00:34<00:26,  1.37s/it]

LassoLarsIC model failed to execute
You are using LassoLarsIC in the case where the number of samples is smaller than the number of features. In this setting, getting a good estimate for the variance of the noise is not possible. Provide an estimate of the noise variance in the constructor.


 79%|███████▊  | 33/42 [00:41<00:04,  1.94it/s]

RANSACRegressor model failed to execute
`min_samples` may not be larger than number of samples: n_samples = 135.


100%|██████████| 42/42 [00:48<00:00,  1.16s/it]


In [195]:
from sklearn.linear_model import OrthogonalMatchingPursuitCV
reg = OrthogonalMatchingPursuitCV(cv=5).fit(tfidf_data, y)

In [200]:
# Load the test data
test_data = pd.read_csv('/kaggle/input/murderdata/Cain_s_Jawbone_Test_data.csv')
test_data['Text'] = test_data['Text'].apply(remove_newlines)
X_test = test_data['Text']
# Preprocess the test data in the same way as the training data
X_test = X_test.str.lower().str.replace(r'[^\w\s]', '')
# Extract features from the test data using the same vectorizer


In [202]:
testt = vectorizer.transform(X_test).toarray()
features = vectorizer.get_feature_names_out()
test_data = pd.DataFrame(testt, columns=features)

In [223]:
y_pred=reg.predict(test_data)

In [224]:
y_pred

array([38.80047906, 19.05921239, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 17.61086534, 38.80047906, 38.80047906, 18.25770521,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       18.12351636, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80047906,
       38.80047906, 38.80047906, 38.80047906, 38.80047906, 38.80

In [225]:
round(17.61086534)

18

In [226]:
pred=[int(i) for i in y_pred]

In [227]:
sub=pd.read_csv("/kaggle/input/murderdata/SampleSubmission.csv")

In [228]:
sub["CorrectPage"] = pred

In [229]:
sub.CorrectPage

0     38
1     19
2     38
3     38
4     38
      ..
70    38
71    38
72    38
73    38
74    38
Name: CorrectPage, Length: 75, dtype: int64

In [231]:
sub.to_csv("sub3.csv", index=False)